# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd
import scipy.stats as stats
from scipy.stats import chi2_contingency#
import numpy as np

In [2]:
df = pd.read_csv("homepage_actions.csv")
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
# duplicates
df.duplicated().value_counts()

False    8188
dtype: int64

In [4]:
# null values
df.isna().value_counts()

timestamp  id     group  action
False      False  False  False     8188
dtype: int64

In [5]:
# Get the total number of viewers
total_viewers = df[df['action'] == 'view']['id'].nunique()

# Get the total number of clickers
total_clickers = df[df['action'] == 'click']['id'].nunique()

# Find viewers who also clicked
viewers_and_clickers = df[df['action'] == 'view']['id'].isin(df[df['action'] == 'click']['id']).sum()

# Find clickers who did not view
clickers_not_viewers = df[df['action'] == 'click']['id'].nunique() - viewers_and_clickers

# Check for overlap between control and experiment groups
control_users = df[df['group'] == 'control']['id'].nunique()
experiment_users = df[df['group'] == 'experiment']['id'].nunique()
overlap = df[df['group'] == 'control']['id'].isin(df[df['group'] == 'experiment']['id']).sum()

# Print the results
print(f"Total viewers: {total_viewers}")
print(f"Total clickers: {total_clickers}")
print(f"Viewers who also clicked: {viewers_and_clickers}")
print(f"Clickers who did not view: {clickers_not_viewers}")
print(f"Overlap between control and experiment groups: {overlap}")

Total viewers: 6328
Total clickers: 1860
Viewers who also clicked: 1860
Clickers who did not view: 0
Overlap between control and experiment groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

#### Hypothesis
H0 : There is no significant difference in user behavior between the experimental homepage and the control group.

H1 : There is significant difference in user behavior between the experimental homepage and the control group.

For this analysis we will use Chi Square test to test whether the experimental homepage was more efffective than that of the control group since we are dealing with categorical variables.

In [6]:
# test statistic
contingency_table = pd.crosstab(df["group"],df["action"])

chi2, p_value, _, _ = chi2_contingency(contingency_table)
print(chi2, p_value)

3.636160051233291 0.056537191086915774


Since the p-value is greater than the alpha level of 0.05, we fail to reject the null hypothesis. This suggests that, based on the chi-square test, there is no significant difference in user behavior between the experimental homepage and the control group.

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [7]:
# Create separate DataFrames for the experiment and control groups
experiment_df = df[df["group"] == "experiment"]
control_df = df[df["group"] == "control"]

# Count the number of clicks in the experiment and control groups
clicks_exp = experiment_df[experiment_df["action"] == "click"]["action"].count()
clicks_ctrl = control_df[control_df["action"] == "click"]["action"].count()

# Calculate the total number of clicks in both groups
total_clicks = clicks_exp + clicks_ctrl

# Calculate the observed click-through rate (CTR) for the experiment group
observed_ctr = clicks_exp / total_clicks

# Print the observed CTR
print("Observed Click-Through Rate for the Experiment Group:", observed_ctr)


Observed Click-Through Rate for the Experiment Group: 0.4989247311827957


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [8]:
p = observed_ctr
n = clicks_exp

# Calculate the variance of the binomial distribution
variance = n * p * (1 - p)

# Calculate the standard deviation of the binomial distribution
std_deviation = np.sqrt(variance)

# Calculate the mean of the binomial distribution
mean = n * p

# Assuming 'popmean' is the expected value or mean you want to compare against
popmean = 0.5

# Calculate the Z-score
z_score = (mean - popmean) / std_deviation

# Print the Z-score
print("Z-score:", z_score)

Z-score: 30.364824004407772


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [9]:
# Calculate the p-value
p_value = 2 * ( 1 - stats.norm.cdf(abs(z_score)))  

# Print the p-value
print("p-value:", p_value)

p-value: 0.0


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

The result from the p-value calculation based on the Z-score closely aligns with the outcome of the previous statistical test using the chi-square test. In both cases, we obtain a very low p-value, which is a strong indicator for rejecting the null hypothesis. 

This consistency in results reinforces our conclusion that there is indeed a significant difference in user behavior between the experimental homepage and the control group. The evidence strongly suggests that the changes made to the homepage had a notable impact on user interaction, making the experimental version more effective.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.